### RAG PIPELINE - Data ingestion to Vector DB pipeline

In [2]:
%pip install langchain_community
%pip install langchain
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter #new version langchain.text_splitter becomes langchain_text_splitters


/Users/adityasingh/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

def process_all_pdfs(pdf_directory):
    """Process all the PDFs in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # find all pdfs (recursive)
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"✔️ Loaded {len(documents)} pages from {pdf_file.name}")

        except Exception as e:
            print(f"❌ Failed to process {pdf_file.name}: {str(e)}")

    return all_documents


In [5]:
all_pdf_documents = process_all_pdfs('../data/')
print(all_pdf_documents)


Found 1 PDF files to process

Processing pdf1.pdf
✔️ Loaded 1 pages from pdf1.pdf
[Document(metadata={'producer': 'Skia/PDF m143 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': '1NH23CD194_Aditya_Kumar', 'source': '../data/pdf/pdf1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}, page_content="Aditya  Kumar  \n 📍Bengaluru,IN 📞+917061939341 ✉seemeadit21824@gmail.com  GitHub  Linkedin\nSummary  Passionate  MERN  stack  developer  with  proﬁciency  in  data  structures  &  algorithms ,  and  basic  machine  learning.  Dedicated  to  building  scalable  web  applications,  optimizing  performance,  and  leveraging  technology  to  create  innovative,  real-world  solutions  that  drive  meaningful  impact.   \nSkills \n●  Programming  Languages :  |  Python  |  C  |  C++  |  Java  |  JavaScript  |  SQL  |  PHP  |  ●  Frontend  Technologies :  |  HTML  |  Bootstrap  |  JS  |  TailwindCSS  |  React  |  ●  Backend  T

In [7]:
all_pdf_documents

[Document(metadata={'producer': 'Skia/PDF m143 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': '1NH23CD194_Aditya_Kumar', 'source': '../data/pdf/pdf1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}, page_content="Aditya  Kumar  \n 📍Bengaluru,IN 📞+917061939341 ✉seemeadit21824@gmail.com  GitHub  Linkedin\nSummary  Passionate  MERN  stack  developer  with  proﬁciency  in  data  structures  &  algorithms ,  and  basic  machine  learning.  Dedicated  to  building  scalable  web  applications,  optimizing  performance,  and  leveraging  technology  to  create  innovative,  real-world  solutions  that  drive  meaningful  impact.   \nSkills \n●  Programming  Languages :  |  Python  |  C  |  C++  |  Java  |  JavaScript  |  SQL  |  PHP  |  ●  Frontend  Technologies :  |  HTML  |  Bootstrap  |  JS  |  TailwindCSS  |  React  |  ●  Backend  Technologies :  |  Flask  |  Node.js  |  Express.js  |  RESTful  API  |  AWS  ●  Da

STEP 2 : Text Splitting get into chunks

In [12]:
def split_documents(documents, chuck_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks...."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chuck_size,
        chunk_overlap=chunk_overlap,
        length_function = len,
        separators=["\n\n", "\n", " ",""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)}")
    if split_docs:
        print(f"\nExample chunk: ")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs

    

In [13]:
chunks = split_documents(all_pdf_documents)
print(chunks)

Split 1 documents into 5

Example chunk: 
Content: Aditya  Kumar  
 📍Bengaluru,IN 📞+917061939341 ✉seemeadit21824@gmail.com  GitHub  Linkedin
Summary  Passionate  MERN  stack  developer  with  proﬁciency  in  data  structures  &  algorithms ,  and  bas...
Metadata: {'producer': 'Skia/PDF m143 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': '1NH23CD194_Aditya_Kumar', 'source': '../data/pdf/pdf1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}
[Document(metadata={'producer': 'Skia/PDF m143 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': '1NH23CD194_Aditya_Kumar', 'source': '../data/pdf/pdf1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}, page_content='Aditya  Kumar  \n 📍Bengaluru,IN 📞+917061939341 ✉seemeadit21824@gmail.com  GitHub  Linkedin\nSummary  Passionate  MERN  stack  developer  with  proﬁciency  in  data  structures  &  algorithm

STEP 3: Embedding (converting text into vector)

In [16]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# from pydantic_core.core_schema import none_schema


class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer"""
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
            Initialize the embedding manager
            Args:
                model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None #later on we are initialize the model
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""

        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded Successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}") #by default dim : 384
        except Exception as e:
            print(f"Error loading model {self.model_name}:{e}")
            raise
    
    def generate_embeddings(self, texts:List[str]) -> np.ndarray:
        """
            Generate embedding for a list of texts

            Args:
                texts:List  of text strings to embed
            Returns:
                numpy array of embeddings with shape(len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not found!!!")
        print(f"Generating embeddings for {len(texts)} texts")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
# initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded Successfully. Embedding dimension: 384


## STEP 4 : Vector Store

In [27]:
import os
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self, collection_name : str="pdf_documents", persist_directory:str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name : Name of the chromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        """Initialize ChromaDB client and collection."""
        try:
            # create persistent chromaDb client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            #Get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"desc":"PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized collection: {self.collection_name}")
            print(f"Existing documents in collection : {self.collection.count()}")

        except Exception as e:
            print(f"Error in initializing vector store: {e}")
            raise
# to add documents

    def add_documents(self, documents : List[Any], embeddings:np.ndarray):
        """
            Add docs and their embeddins in the vector store

            Args:
                documents : List of langchain documents
                embeddings : Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)} documents to vector store")
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i,(doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generating unique Id
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            documents_text.append(doc.page_content)

            embeddings_list.append(embedding.tolist())
    # Add to collection
        try:
            self.collection.add(
                ids = ids, 
                embeddings = embeddings_list,
                metadatas = metadatas,
                documents = documents_text
            )
            print(f"Successfully Added {len(documents)} documents to vector store")
            print(f"Total documents in collection : {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector Store: {e} ")
            raise

In [28]:
vectorstore = VectorStore()
vectorstore

Vector store initialized collection: pdf_documents
Existing documents in collection : 0


In [29]:
### convert the text to embeddings
texts = [doc.page_content for doc in chunks]

### Generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)

### store in the vector database
vectorstore.add_documents(chunks, embeddings)



Generating embeddings for 5 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]

Generated embeddings with shape: (5, 384)
Adding 5 documents to vector store
Successfully Added 5 documents to vector store
Total documents in collection : 5
